In [71]:
import sc2reader
from sc2reader.engine.plugins import APMTracker, ContextLoader, SelectionTracker
from sc2reader import events, data
import pandas as pd
import glob
from IPython.display import display
import json
from collections import Counter

pd.options.display.max_columns = None

In [72]:
class HandleReplays:
    '''
    A class to handle all thing SC2 replay.

    ...

    Attributes
    ----------
    attr_map : dict
        Attribute map
    unit_map : dict
        Unit map

    Methods
    -------
    load_replays(glob_path, amount=None, verbose=True):
        Loads SC2 replays found in path.
    '''


    def __init__(self, attr_map=None, unit_map=None):
        '''
        Constructs all the necessary attributes for HandleReplays.

        Parameters
        ----------
        attr_map : dict
            Attribute map
        unit_map : dict
            Unit map
        '''

        if attr_map is None:
            return Exception('Please provide an attr_map.')
        if unit_map is None:
            return Exception('Please provide a unit_map.')
        
        self.attr_map = attr_map
        self.unit_map = unit_map


    def load_replays(self, glob_path, amount=None, verbose=True):
        '''
        Loads SC2 replays found in the provided path.

        If the argument 'amount' is passed, then only that amount will be loaded.

        Parameters
        ----------
        glob_path : str
            Path to .SC2Replay files as a glob string
        amount : int, optional
            Number of replays to be loaded (default is All)
        verbose : bool, optional
            Show verbose information (default is True)
            

        Returns
        -------
        None
        '''

        replays = []

        paths = [path for path in glob.glob(glob_path, recursive=True)]
        loader_amount = len(paths) if amount is None or amount > len(paths) else amount

        for i, path in enumerate(paths[:amount]):
            if verbose:
                print('\rLoading replay {:4}/{:04} | Loaded {:6.2f}% of total!'.format(i+1, loader_amount, (i+1)/loader_amount*100), end='', flush=True)
            
            replay = sc2reader.load_replay(path, engine=sc2reader.engine.GameEngine(plugins=[ContextLoader(), APMTracker(), SelectionTracker()]))
            replays.append(replay)
        
        self.replays = replays


    def get_dataframe(self, matchup, verbose=True):
        '''
        Returns the generated DataFrame with the provided matchup.

        Parameters
        ----------
        matchup : str
            Matchup as a two character string with membership [PP,PT,TP,PZ,ZP,TT,TZ,ZT,ZZ].
        verbose : bool, optional
            Show verbose information (default is True)
            

        Returns
        -------
        Matchup DataFrame
        '''

        race_map = {
            'P': 'Protoss',
            'T': 'Terran',
            'Z': 'Zerg'
        }
        
        if matchup is None or len(matchup) != 2 or matchup.upper()[0] not in race_map or matchup.upper()[1] not in race_map:
            return Exception('The parameter "matchup" must be a string containing the initials of each race of the matchup, e.g.: "TT" or "PT".')
        
        races = [matchup.upper()[0], matchup.upper()[1]]
        df_data = []
        valid_games = 0
        pt_dict = dict.fromkeys(self.unit_map[race_map[races[0]]] + self.unit_map[race_map[races[1]]], 0)

        for i, replay in enumerate(self.replays[:]):

            # only if it's the matchup we're looking for
            if len(set([replay.players[0].pick_race[0], replay.players[1].pick_race[0]]) & set(races)) == 2:
                if verbose:
                    print('\n{} Game #{:03} | {} vs. {} {}'.format('-'*17, i+1, replay.players[0].pick_race, replay.players[1].pick_race, '-'*17))

                valid_games += 1
                dd = {}

                for event in replay.events:
                    # break if nothing to collect
                    if isinstance(event, events.PlayerLeaveEvent):
                        if verbose:
                            print('Player {} left {} seconds into the game.'.format(event.player, event.second))
                        break

                    # every 30 seconds
                    if event.second % 30 == 0:

                        # every 10 seconds
                        if isinstance(event, events.PlayerStatsEvent):
                            d = {}

                            is_player_1 = replay.players[1].pid == event.pid
                            race = replay.players[is_player_1].pick_race[0]
                            win = replay.players[is_player_1].result == 'Win'

                            lower_bound = 0 if event.second == 0 else event.second-30
                            ap30s = sum(list(replay.players[is_player_1].aps.values())[lower_bound:event.second])

                            d['match_id'] = i
                            d['frame'] = event.frame
                            d['second'] = event.second
                            d['race'] = race
                            d['ap30s'] = ap30s

                            for attr in self.attr_map['PlayerStatsEvent']:
                                d[attr] = eval('event.' + attr)
                            
                            d['win'] = win

                            dd[race] = d

                        # every 15 seconds
                        if isinstance(event, events.UnitPositionsEvent):
                            dd1 = dd[races[0]]
                            dd2 = dd[races[1]]

                            dd1.update(pt_dict)
                            dd2.update(pt_dict)

                            current_units = [str(a).split(' ')[0].lower() for a in event.units.keys()]
                            counted_units = Counter(current_units)

                            for k in counted_units:
                                if k in self.unit_map[race_map[races[0]]]:
                                    dd1[k] = counted_units[k]
                                elif k in self.unit_map[race_map[races[1]]]:
                                    dd2[k] = counted_units[k]
                                elif verbose:
                                    print('Found invalid unit "{}".'.format(k))

                            df_data.extend([dd1, dd2])
                            dd = {}

        df = pd.DataFrame(df_data)
        if verbose:
            print('\nEND: ({}, {}) found {} valid games out of {}.'.format(*df.shape, valid_games, len(self.replays)))

        return df

In [73]:
with open('./stats.json', 'rb') as f:
    attr_map = json.load(f)

unit_data = json.loads(data.unit_data)

unit_map = {}
for k in unit_data:
    unit_map[k] = list(unit_data[k].keys())

In [74]:
hr = HandleReplays(attr_map=attr_map, unit_map=unit_map)

hr.load_replays('./_data/**/*.SC2Replay', amount=10)

Loading replay   10/0010 | Loaded 100.00% of total!

In [75]:
matchup = 'PZ'
df = hr.get_dataframe(matchup)

## Uncomment to save as CSV
# valid_matches = len(set(df.loc[:,'match_id']))
# df.to_csv('./_sc2_{}_{}{}.csv'.format(valid_matches, *list(matchup)))


----------------- Game #008 | Protoss vs. Zerg -----------------
Player Player 2 - RiSky (Zerg) left 856 seconds into the game.

----------------- Game #009 | Zerg vs. Protoss -----------------
Found invalid unit "lurkerburrowed".
Found invalid unit "lurkerburrowed".
Found invalid unit "lurkerburrowed".
Found invalid unit "lurkerburrowed".
Found invalid unit "broodlingescort".
Found invalid unit "lurkerburrowed".
Player Player 2 - Probe (Protoss) left 3736 seconds into the game.

----------------- Game #010 | Zerg vs. Protoss -----------------
Player Player 1 - RiSky (Zerg) left 1424 seconds into the game.

END: (302, 116) found 3 valid games out of 10.


In [76]:
df.describe()

,match_id,frame,second,ap30s,minerals_current,vespene_current,minerals_collection_rate,vespene_collection_rate,workers_active_count,minerals_used_in_progress,vespene_used_in_progress,minerals_used_current,vespene_used_current,minerals_lost,vespene_lost,minerals_killed,vespene_killed,food_used,food_made,minerals_used_active_forces,vespene_used_active_forces,adept,archon,assimilator,carrier,colossus,cyberneticscore,darkshrine,darktemplar,disruptor,fleetbeacon,forge,gateway,hightemplar,immortal,interceptor,mothership,mothershipcore,nexus,observer,oracle,phoenix,photoncannon,probe,pylon,reactor,roboticsbay,roboticsfacility,sentry,stalker,stargate,tempest,templararchive,twilightcouncil,voidray,warpgate,warpprism,warpprismphasing,zealot,baneling,banelingburrowed,banelingcocoon,banelingnest,broodling,broodlord,broodlordcocoon,corruptor,creeptumor,creeptumorburrowed,drone,droneburrowed,evolutionchamber,extractor,greaterspire,hatchery,hive,hydralisk,hydraliskburrowed,hydraliskden,infestationpit,infestedterran,infestedterranburrowed,infestor,infestorburrowed,lair,locust,lurker,lurkerden,mutalisk,nydusnetwork,nydusworm,overlord,overseer,overseercocoon,queen,queenburrowed,ravager,roach,roachburrowed,roachwarren,spawningpool,spinecrawler,spinecrawleruprooted,spire,sporecrawler,sporecrawleruprooted,swarmhost,swarmhostburrowed,ultralisk,ultraliskburrowed,ultraliskcavern,viper,zergling,zerglingburrowed
count,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.0,302.000000,302.000000,302.0,302.0,302.000000,302.0,302.0,302.0,302.0,302.000000,302.000000,302.000000,302.000000,302.0,302.0,302.000000,302.000000,302.000000,302.0,302.000000,302.0,302.0,302.0,302.0,302.000000,302.000000,302.0,302.000000,302.0,302.0,302.000000,302.0,302.000000,302.000000,302.000000,302.000000,302.0,302.0,302.0,302.000000,302.000000,302.0,302.000000,302.0,302.0,302.000000,302.0,302.0,302.0,302.0,302.0,302.0,302.000000,302.0,302.0,302.0,302.0,302.0,302.000000,302.000000,302.0,302.0,302.000000,302.0,302.000000,302.0,302.0,302.000000,302.000000,302.0,302.000000,302.0,302.000000,302.000000,302.0,302.0,302.0,302.000000,302.0,302.0,302.000000,302.000000,302.0,302.0,302.0,302.0,302.0,302.000000,302.000000,302.0
mean,8.139073,26174.304636,1635.894040,184.062914,1694.950331,815.158940,2003.506623,744.549669,57.304636,733.278146,239.486755,17464.056291,4575.993377,19364.884106,6332.748344,19627.996689,6367.384106,160.201987,212.423841,5711.738411,3792.384106,0.086093,0.374172,0.0,0.168874,0.036424,0.0,0.0,0.102649,0.0,0.0,0.0,0.0,0.264901,0.139073,3.076159,0.033113,0.0,0.0,0.013245,0.039735,0.033113,0.0,0.165563,0.0,0.0,0.0,0.0,0.165563,0.728477,0.0,0.066225,0.0,0.0,0.149007,0.0,0.016556,0.009934,0.784768,0.241722,0.0,0.0,0.0,0.135762,0.185430,0.0,1.135762,0.0,0.0,0.119205,0.0,0.0,0.0,0.0,0.0,0.0,0.009934,0.0,0.0,0.0,0.0,0.0,0.129139,0.056291,0.0,0.0,0.003311,0.0,0.175497,0.0,0.0,0.089404,0.119205,0.0,0.125828,0.0,0.476821,0.188742,0.0,0.0,0.0,0.006623,0.0,0.0,0.079470,0.006623,0.0,0.0,0.0,0.0,0.0,0.135762,3.317881,0.0
std,0.541190,16342.386017,1021.399126,70.970725,1852.521367,706.777988,935.938344,460.110512,23.831773,545.637652,318.248923,6914.974328,3016.506198,18013.480389,5909.255858,18268.536917,5945.241539,47.335383,86.632376,3251.145677,2693.803634,0.363453,1.038653,0.0,1.091172,0.248580,0.0,0.0,0.453183,0.0,0.0,0.0,0.0,0.952018,0.672574,13.806355,0.179228,0.0,0.0,0.114512,0.211961,0.213100,0.0,0.957089,0.0,0.0,0.0,0.0,0.873611,3.125150,0.0,0.319237,0.0,0.0,0.729754,0.0,0.127813,0.099337,2.184976,1.556632,0.0,0.0,0.0,1.017184,0.895941,0.0,4.162574,0.0,0.0,0.690968,0.0,0.0,0.0,0.0,0.0,0.0,0.172631,0.0,0.0,0.0,0.0,0.0,0.794423,0.315928,0.0,0.0,0.057544,0.0,1.358740,0.0,0.0,0.530110,0.422414,0.0,0.458289,0.0,1.738118,1.129845,0.0,0.0,0.0,0.081244,0.0,0.0,0.461446,0.081244,